# Get Data
This notebook includes the procedures for fetching:
* The DC Fast Charging ("DCFC") locations from NREL servers via NREL's APIs, and
* The North Carolina major road network, from OpenStreetMap via the osmnx package.

In [3]:
#Import packages
import os, shutil, requests
import osmnx as ox
import pandas as pd
import geopandas as gpd

## 1. Fetch the DCFC data
* Fetch the data, which includes latitude and longitude fields, filtered for NC into a Pandas dataframe.
* Clean the data: remove unneeded columns and fiter for non-Tesla public charging records.
* Convert to a geodataframe, save as a shapefile.

In [1]:
#Get the NREL API key (stored as local file)
key = open('Data/NRELkey.txt','r').readline()

In [2]:
#Construct the service request
baseURL = 'https://developer.nrel.gov/api/alt-fuel-stations/v1.json?'
params = {
    'api_key':key,                #Our key, to authenticate the service
    'status':'E',                 #Request only currently open sites 
    'fuel_type':'ELEC',           #Request only Electric sites
    'ev_charging_level':'dc_fast',#Request only DCFC sites
    'state':'NC' ,                #Request only NC sites
    'access':'public',            #Request only Public sites
    'groups_with_access_code':'Public', #Request only sites open to Public
    'limit':'all'                 #Don't cap results to first 200 (default)
}

In [16]:
#Execute the request, converting the response to a dataframe
df_ALL = pd.DataFrame.from_dict(requests.get(baseURL,params).json()['fuel_stations'])
print(f'{len(df_ALL)} records retrieved')

86 records retrieved


In [5]:
#Drop excess columns
dropCols = [col for col in df_ALL.columns if col[:3] in ('bd_','cng','hy_','ng_','e85','lng','lpg')]
dropCols += [col for col in df_ALL.columns if col[-2:] == 'fr']
df_ALL.drop(columns = dropCols,inplace=True)

In [6]:
#Remove sites in the Tesla network
df_DCFC = df_ALL.loc[df_ALL['ev_network'] != 'Tesla'].reset_index()

In [11]:
#Create a folder to hold NREL data
if not os.path.exists('Data/NREL'):
    print("Creating NREL folder in Data directory")
    os.mkdir('Data/NREL')
else:
    print("Using existing NREL folder in Data directory")

Creating NREL folder in Data directory


In [15]:
#Save to a csv
df_DCFC.to_csv('Data/NREL/DCFCsites.csv',index=False)
print(f"{len(df_DCFC)} records save to Data/NREL/DCFCsites.csv")

68 records save to Data/NREL/DCFCsites.csv


In [8]:
#Convert to a geodataframe
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_DCFC.longitude, df_DCFC.latitude)]
gdf_DCFC = gpd.GeoDataFrame(df_DCFC,geometry=geometry,crs='epsg:4326')

In [17]:
gdf_DCFC[['id','geometry']].to_file('Data/NREL/NC_DCFC.shp')
print(f"{len(df_DCFC)} features stored as Data/NREL/NC_DCFC.shp")

68 features stored as Data/NREL/NC_DCFC.shp


## 2. Fetch NC major roads as a graph and shapefile
* Read NC major roads (motorway, primary) graphml into a graph object; fetch if not present

In [18]:
#Create a folder to hold OSM data, if not there already
if not os.path.exists('Data/OSM'):
    print("Creating OSM folder in Data directory")
    os.mkdir('Data/OSM')
else:
    print("Using existing OSM folder in Data directory")

Creating OSM folder in Data directory


In [19]:
#Fetch the NC highway graph
graph_filename = 'Data/OSM/NC_highways_all.graphml'
if os.path.exists(graph_filename):
    print(f"Loading graph from {graph_filename}")
    nc_graph = ox.load_graphml(filename=os.path.basename(graph_filename),
                               folder=os.path.dirname(graph_filename))
else:
    print("Fetching data. This may take a while...")
    nc_graph = ox.graph_from_place("North Carolina",
                                    network_type='drive',
                                    simplify=True,
                                    retain_all=False,
                                    truncate_by_edge=True,
                                    infrastructure='way["highway"~"motorway|trunk|primary"]')
    print(f"Saving file to {graph_filename}")
    ox.save_graphml(nc_graph,
                    filename=os.path.basename(graph_filename),
                    folder=os.path.dirname(graph_filename))
                    

Fetching data. This may take a while...
Saving file to Data/OSM/NC_highways_all.graphml


In [20]:
#Save as a shapefile
ox.save_graph_shapefile(G=nc_graph,filename="NC_highways",folder="Data/OSM")